# Narrative Generation Demo

This notebook demonstrates how to use the narrative generation feature with `factual_explanation.plot(style='narrative', ...)`

In [19]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from calibrated_explanations import CalibratedExplainer, __version__
print("Calibrated Explanations version:", __version__)

Calibrated Explanations version: v0.9.1


## 1. Prepare Data and Train Model

In [20]:
# Load iris dataset (binary classification)
X, y = load_iris(return_X_y=True)
mask = y < 2  # Only use classes 0 and 1
X, y = X[mask], y[mask]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Train model
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 70
Test set size: 30


## 2. Create Calibrated Explainer

In [21]:
# Create explainer with feature names
explainer = CalibratedExplainer(
    model, 
    X_train, 
    y_train,
    feature_names=["sepal_length", "sepal_width", "petal_length", "petal_width"]
)

print("Explainer created successfully!")

Explainer created successfully!


## 3. Generate Factual Explanations

In [22]:
# Generate factual explanations for first 5 test instances
factual_explanation = explainer.explain_factual(X_test[:5])

print(f"Generated explanations for {len(factual_explanation.explanations)} instances")

Generated explanations for 5 instances


## 4. Generate Narratives - DataFrame Output

**This is the main feature!** Generate human-readable narratives for all expertise levels:

In [23]:
# Generate narratives as DataFrame
result = factual_explanation.to_narrative(
    template_path="explain_template.yaml",
    expertise_level=("beginner", "intermediate", "advanced"),
    output_format="dataframe"
)

print("Result type:", type(result))
print("\nColumns:", result.columns.tolist())
print("\nDataFrame shape:", result.shape)

# Display the DataFrame
result

Result type: <class 'pandas.core.frame.DataFrame'>

Columns: ['instance_index', 'factual_explanation_beginner', 'factual_explanation_intermediate', 'factual_explanation_advanced', 'expertise_level', 'problem_type']

DataFrame shape: (5, 6)


,instance_index,factual_explanation_beginner,factual_explanation_intermediate,factual_explanation_advanced,expertise_level,problem_type
0,0,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974\n...,"(beginner, intermediate, advanced)",binary_classification
1,1,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974\n...,"(beginner, intermediate, advanced)",binary_classification
2,2,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974 [...,Prediction: 1\nCalibrated Probability: 0.974\n...,"(beginner, intermediate, advanced)",binary_classification
3,3,Prediction: 1\nCalibrated Probability: 0.029 [...,Prediction: 1\nCalibrated Probability: 0.029 [...,Prediction: 1\nCalibrated Probability: 0.029\n...,"(beginner, intermediate, advanced)",binary_classification
4,4,Prediction: 1\nCalibrated Probability: 0.029 [...,Prediction: 1\nCalibrated Probability: 0.029 [...,Prediction: 1\nCalibrated Probability: 0.029\n...,"(beginner, intermediate, advanced)",binary_classification


## 5. View Individual Narratives

In [24]:
# View beginner-level narrative for first instance
print("Beginner Narrative for Instance 0:")
print("=" * 80)
print(result.iloc[0]['factual_explanation_beginner'])

Beginner Narrative for Instance 0:
Prediction: 1
Calibrated Probability: 0.974 [0.974, 1.000]

Factors increasing Calibrated Probability:
* petal_length (5.1) > 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]
* petal_width (1.6) > 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]
* sepal_width (2.7) <= 3.15, [⚠️ highly uncertain, ⚠️ direction uncertain]

Factors decreasing Calibrated Probability:


In [25]:
# View advanced-level narrative for first instance
print("Advanced Narrative for Instance 0:")
print("=" * 80)
print(result.iloc[0]['factual_explanation_advanced'])

Advanced Narrative for Instance 0:
Prediction: 1
Calibrated Probability: 0.974
Prediction Interval: [0.974, 1.000]

Factors increasing Calibrated Probability:
petal_length (5.1) > 2.45 — weight ≈ 0.474 [-0.026, 0.974], [⚠️ highly uncertain, ⚠️ direction uncertain]
petal_width (1.6) > 0.80 — weight ≈ 0.474 [-0.026, 0.974], [⚠️ highly uncertain, ⚠️ direction uncertain]
sepal_width (2.7) <= 3.15 — weight ≈ 0.158 [-0.026, 0.325], [⚠️ highly uncertain, ⚠️ direction uncertain]

Factors decreasing Calibrated Probability:


## 6. Alternative Output Formats

In [26]:
# Text output - formatted for console/reports
text_result = factual_explanation.plot(
    style="narrative",
    expertise_level="intermediate",
    output="text"
)

print(text_result)


Instance 0

Factual Explanation (Intermediate):
--------------------------------------------------------------------------------
Prediction: 1
Calibrated Probability: 0.974 [0.974, 1.000]

Factors increasing Calibrated Probability:
* petal_length (5.1) > 2.45 — weight ≈ 0.474, [⚠️ highly uncertain, ⚠️ direction uncertain]
* petal_width (1.6) > 0.80 — weight ≈ 0.474, [⚠️ highly uncertain, ⚠️ direction uncertain]
* sepal_width (2.7) <= 3.15 — weight ≈ 0.158, [⚠️ highly uncertain, ⚠️ direction uncertain]

Factors decreasing Calibrated Probability:

Instance 1

Factual Explanation (Intermediate):
--------------------------------------------------------------------------------
Prediction: 1
Calibrated Probability: 0.974 [0.974, 1.000]

Factors increasing Calibrated Probability:
* petal_length (4.0) > 2.45 — weight ≈ 0.474, [⚠️ highly uncertain, ⚠️ direction uncertain]
* petal_width (1.3) > 0.80 — weight ≈ 0.474, [⚠️ highly uncertain, ⚠️ direction uncertain]
* sepal_width (2.3) <= 3.15 — we

In [27]:
# Dict output - for programmatic access
dict_result = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",
    output="dict"
)

print("Dict output - first instance:")
print(dict_result[0])

Dict output - first instance:
{'instance_index': 0, 'factual_explanation_beginner': 'Prediction: 1\nCalibrated Probability: 0.974 [0.974, 1.000]\n\nFactors increasing Calibrated Probability:\n* petal_length (5.1) > 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]\n* petal_width (1.6) > 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]\n* sepal_width (2.7) <= 3.15, [⚠️ highly uncertain, ⚠️ direction uncertain]\n\nFactors decreasing Calibrated Probability:', 'expertise_level': ('beginner',), 'problem_type': 'binary_classification'}


In [28]:
# HTML output - for web display
from IPython.display import HTML

html_result = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",
    output="html"
)

HTML(html_result)

instance_index,factual_explanation_beginner,expertise_level,problem_type
0,"Prediction: 1Calibrated Probability: 0.974 [0.974, 1.000]Factors increasing Calibrated Probability:* petal_length (5.1) > 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_width (1.6) > 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]* sepal_width (2.7) <= 3.15, [⚠️ highly uncertain, ⚠️ direction uncertain]Factors decreasing Calibrated Probability:",beginner,binary_classification
1,"Prediction: 1Calibrated Probability: 0.974 [0.974, 1.000]Factors increasing Calibrated Probability:* petal_length (4.0) > 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_width (1.3) > 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]* sepal_width (2.3) <= 3.15, [⚠️ highly uncertain, ⚠️ direction uncertain]Factors decreasing Calibrated Probability:",beginner,binary_classification
2,"Prediction: 1Calibrated Probability: 0.974 [0.974, 1.000]Factors increasing Calibrated Probability:* petal_length (4.8) > 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_width (1.8) > 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]Factors decreasing Calibrated Probability:",beginner,binary_classification
3,"Prediction: 1Calibrated Probability: 0.029 [0.000, 0.029]Factors increasing Calibrated Probability:* sepal_width (3.0) <= 3.15, [⚠️ direction uncertain]Factors decreasing Calibrated Probability:* sepal_length (4.8) <= 5.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_length (1.4) <= 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_width (0.3) <= 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]",beginner,binary_classification
4,"Prediction: 1Calibrated Probability: 0.029 [0.000, 0.029]Factors increasing Calibrated Probability:* sepal_length (5.1) <= 5.45, [⚠️ direction uncertain]* sepal_width (3.8) > 3.15, [⚠️ direction uncertain]Factors decreasing Calibrated Probability:* petal_length (1.9) <= 2.45, [⚠️ highly uncertain, ⚠️ direction uncertain]* petal_width (0.4) <= 0.80, [⚠️ highly uncertain, ⚠️ direction uncertain]",beginner,binary_classification


## 7. Single Expertise Level

In [29]:
# Generate only beginner-level narratives
beginner_only = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",  # Single level as string
    output="dataframe"
)

print("Columns:", beginner_only.columns.tolist())
beginner_only

Columns: ['instance_index', 'factual_explanation_beginner', 'expertise_level', 'problem_type']


,instance_index,factual_explanation_beginner,expertise_level,problem_type
0,0,Prediction: 1\nCalibrated Probability: 0.974 [...,"(beginner,)",binary_classification
1,1,Prediction: 1\nCalibrated Probability: 0.974 [...,"(beginner,)",binary_classification
2,2,Prediction: 1\nCalibrated Probability: 0.974 [...,"(beginner,)",binary_classification
3,3,Prediction: 1\nCalibrated Probability: 0.029 [...,"(beginner,)",binary_classification
4,4,Prediction: 1\nCalibrated Probability: 0.029 [...,"(beginner,)",binary_classification


## Summary

**Key Points:**
- Use `factual_explanation.plot(style="narrative", ...)` to generate narratives
- Supports 3 expertise levels: `"beginner"`, `"intermediate"`, `"advanced"`
- Supports 4 output formats: `"dataframe"`, `"text"`, `"html"`, `"dict"`
- Works with factual, alternative, and fast explanations
- Automatically detects problem type (classification, regression, etc.)